# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [2]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [93]:
# Seleccionar el recurso a descargar
resource_url = "https://en.wikipedia.org/wiki/List_of_most-streamed_songs_on_Spotify"
# Para evitar que el servidor bloquee la peticion
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
# Petición para descargar el html de wikipedia
response = requests.get(resource_url, headers=headers)

# Si la petición se ha ejecutado correctamente (código 200), entonces el contenido HTML de la página se ha podido descargar
if response:
    # Transformamos el HTML plano en un LXML (estructurado y anidado, con forma de árbol)
    soup = BeautifulSoup(response.content, 'html')
    soup

In [ ]:
#Selecionamos la primera tabla de la pagina que contiene las canciones
tables = soup.find_all('table')
table = tables[0].find('tbody')
table

In [94]:
def clean_text(x):
    datos = x.find_all(['td','th'])
    for i in range(len(datos)):
        aux=datos[i].text.strip()
        if "$" in aux:
            aux=datos[i].replace('$', '')
        
        datos[i]=aux
    return datos

rows = table.find_all('tr')
headers = rows[0:1]
tuples = rows[1:]
headers = list(map(clean_text, headers))
tuples = list(map(clean_text, tuples))
rows

[<tr>
 <th>Rank
 </th>
 <th>Song
 </th>
 <th>Artist(s)
 </th>
 <th>Streams<br/>(billions)
 </th>
 <th>Release date
 </th>
 <th class="unsortable"><abbr title="References">Ref.</abbr>
 </th></tr>,
 <tr>
 <td style="text-align:center;">1</td>
 <th scope="row">"<a href="/wiki/Blinding_Lights" title="Blinding Lights">Blinding Lights</a>"
 </th>
 <td style="text-align:left;">The Weeknd</td>
 <td style="text-align:center;">5.276</td>
 <td style="text-align:right;">29 November 2019</td>
 <td style="text-align:center;"></td></tr>,
 <tr>
 <td style="text-align:center;">2</td>
 <th scope="row">"<a href="/wiki/Shape_of_You" title="Shape of You">Shape of You</a>"
 </th>
 <td style="text-align:left;">Ed Sheeran</td>
 <td style="text-align:center;">4.768</td>
 <td style="text-align:right;">6 January 2017</td>
 <td style="text-align:center"></td></tr>,
 <tr>
 <td style="text-align:center;">3</td>
 <th scope="row">"<a href="/wiki/Sweater_Weather" title="Sweater Weather">Sweater Weather</a>"
 </th>
 <t

In [96]:
df = pd.DataFrame(tuples, columns=headers[0])
df = df.iloc[:, :-1]
df = df.replace('', pd.NA)
df = df.dropna(how='any')
df

,Rank,Song,Artist(s),Streams(billions),Release date
0,1,"""Blinding Lights""",The Weeknd,5.276,29 November 2019
1,2,"""Shape of You""",Ed Sheeran,4.768,6 January 2017
2,3,"""Sweater Weather""",The Neighbourhood,4.379,3 December 2012
4,5,"""As It Was""",Harry Styles,4.246,1 April 2022
5,6,"""Someone You Loved""",Lewis Capaldi,4.223,8 November 2018
...,...,...,...,...,...
90,91,"""Good 4 U""",Olivia Rodrigo,2.582,14 May 2021
92,93,"""Don't Stop Me Now""",Queen,2.565,26 January 1979
96,97,"""Dreams""",Fleetwood Mac,2.544,4 February 1977
98,99,"""Someone Like You""",Adele,2.523,24 January 2011
